## This is my exploration of Keras models. See second half for non-sequential.

In [19]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import *
import numpy as np

In [13]:
model = Sequential()

# Dimension: (288, 5, 1)
model.add(TimeDistributed(Conv2D(filters=10,
								 kernel_size=(9, 2),
								 strides=(1, 1),
								 padding='SAME',
								 activation='relu',
								 input_shape=(288, 5, 1))))
# Dimension: (288, 5, 10)
model.add(TimeDistributed(MaxPooling2D(pool_size=(4, 2), strides=(2, 1))))
# Dimension: (144, 4, 10)
model.add((LSTM(1, input_shape=(141, 3, 20), return_sequences=True)))
# Dimension: (144, 4, 10)
model.add(TimeDistributed(Conv2D(20, kernel_size=(5, 2), strides=(1, 1), activation='relu')))
# Dimension: (140, 3, 20)
model.add(TimeDistributed(MaxPooling2D(pool_size=(4, 1), strides=(4, 1))))
# Dimension: (35, 3, 20)
model.add((LSTM(1, input_shape=(35, 3, 20), return_sequences=True)))
# Dimension: (35, 3, 20)
model.add(TimeDistributed(Flatten()))
# Dimension: (2100)
model.add(TimeDistributed(Dense(1000)))
model.add(TimeDistributed(Dense(500)))
model.add(Dense(88))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [20]:
import os
cqt_slice_filenames = sorted(os.listdir("sample-cqt-aitan\\sample-cqt-aitan"))
pr_slice_filenames = sorted(os.listdir("sample-pianoroll-aitan\\sample-pianoroll-aitan"))
num_slices = len(cqt_slice_filenames)
assert(len(cqt_slice_filenames) == len(pr_slice_filenames))

In [28]:
# # Prepare x and y data
# x_train = np.zeros((1, num_slices, 288, 5, 1))
# y_train = np.zeros((1, num_slices, 88, 1))
# for i in range(num_slices):
#     cqt_slice = np.fromfile("sample-cqt-aitan\\sample-cqt-aitan\\" + cqt_slice_filenames[i]).reshape((288, 5, 1))
#     pr_slice = np.fromfile("sample-pianoroll-aitan\\sample-pianoroll-aitan\\" + pr_slice_filenames[i]).reshape((88, 1))
#     x_train[0, i, :, :, :] = cqt_slice
#     y_train[0, i, :, :] = pr_slice

In [ ]:
model.fit(x_train, y_train, epochs=5)

In [ ]:
score, acc = model.evaluate(x_train, y_train)

## Abort and try non-sequential

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
# Prepare x and y data
x = np.zeros((num_slices, 288, 5, 1))
y = np.zeros((num_slices, 88))
for i in range(num_slices):
    cqt_slice = np.fromfile("sample-cqt-aitan\\sample-cqt-aitan\\" + cqt_slice_filenames[i]).reshape((288, 5, 1))
    pr_slice = np.fromfile("sample-pianoroll-aitan\\sample-pianoroll-aitan\\" + pr_slice_filenames[i]).reshape((88))
    x[i, :, :, :] = cqt_slice
    y[i, :] = pr_slice

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [62]:
model = Sequential()

# Dimension: (288, 5, 1)
model.add(Conv2D(filters=10,
                 kernel_size=(9, 2),
                 strides=(1, 1),
                 padding='SAME',
                 activation='relu',
                 input_shape=(288, 5, 1)))
# Dimension: (288, 5, 10)
model.add(MaxPooling2D(pool_size=(4, 2), strides=(2, 1)))
# Dimension: (143, 4, 10)
model.add(Conv2D(20, kernel_size=(5, 2), strides=(1, 1), activation='relu'))
# Dimension: (139, 3, 20)
model.add(MaxPooling2D(pool_size=(4, 1), strides=(4, 1)))
# Dimension: (34, 3, 20)
model.add(Flatten())
# Dimension: (2100)
model.add(Dense(1000))
model.add(Dense(500))
model.add(Dense(88))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 288, 5, 10)        190       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 143, 4, 10)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 139, 3, 20)        2020      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 34, 3, 20)         0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 2040)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 1000)              2041000   
_________________________________________________________________
dense_32 (Dense)             (None, 500)             

In [63]:
model.fit(x_train[:10], y_train[:10], epochs=5)

Epoch 1/5
10/10 [==============================] - 0s 15ms/step - loss: 0.6993 - accuracy: 0.4659
Epoch 2/5
10/10 [==============================] - 0s 2ms/step - loss: 0.5123 - accuracy: 0.9864
Epoch 3/5
10/10 [==============================] - 0s 2ms/step - loss: 0.3332 - accuracy: 1.0000
Epoch 4/5
10/10 [==============================] - 0s 2ms/step - loss: 0.1688 - accuracy: 1.0000
Epoch 5/5
10/10 [==============================] - 0s 2ms/step - loss: 0.0626 - accuracy: 1.0000


In [64]:
score, acc = model.evaluate(x_test, y_test)

97/97 [==============================] - 0s 483us/step


In [65]:
acc

1.0